In [1]:
cd ..

/home/amit/Practice/Semantic-search


In [2]:
#Import standard modules
%run lib/__init__.py
%matplotlib inline


## Set up Mongo Database on AWS instance

- AWS E2 t2.micro instance running on public ip=35.174.105.75
- Ensure port 27016 open for connecting to MongoDB
- Docker Mongo image 'loaded 
- Docker container created, name = 'this_mongo'
- Ensure Docker container running

In [3]:
import pymongo

Set up link to Mongo client on AWS

In [4]:
IP='35.174.105.75'

In [5]:
client = pymongo.MongoClient(IP, 27016)

Check existing databases

In [6]:
client.database_names()

['admin', 'local', 'myWiki']

Create a reference to a new database for wikipedia information ( lets refer to database refs as _dbr)

In [7]:
myWiki_dbr=client.myWiki

Create a reference to a test collection  ( lets refer to collection refs with _cllr)

In [8]:
test_cllr=myWiki_dbr.test_collection

Add a test record to create db and collection

In [9]:
test_record={ 'message' : 'Test db creation'}
test_cllr.insert_one(test_record)

# Maintenance  / management of DB content 

## Check dbs and collections set up

In [10]:
client.database_names() 

['admin', 'local', 'myWiki']

In [34]:
myWiki_dbr.collection_names()

['pagetestload_collection',
 'test_collection',
 'loads_collection',
 'page_collection',
 'pagetest_collection',
 'category_collection',
 'pageload_collection']

## Check documents inserted
- Assign a query to a cursor
- run query by calling cursor
- Use cursor to iterate through query results

In [13]:
cursor=test_cllr.find()
cursor
docs=list(cursor)
docs

[{'_id': ObjectId('5a108b2b05f1b300763b37b7'), 'message': 'Test db creation'},
 {'_id': ObjectId('5a108f0005f1b300763b37b8'), 'message': 'Test db creation'},
 {'_id': ObjectId('5a15b7cb05f1b301cca4630e'), 'message': 'Test db creation'},
 {'_id': ObjectId('5a22139dcd7a8902450cc3e0'), 'message': 'Test db creation'},
 {'_id': ObjectId('5a22e630cd7a8903341027f8'), 'message': 'Test db creation'},
 {'_id': ObjectId('5a68febfcd7a890148cbef10'), 'message': 'Test db creation'},
 {'_id': ObjectId('5a69000dcd7a890148cbef12'), 'message': 'Test db creation'},
 {'_id': ObjectId('5a690f16cd7a8901dc5244ac'), 'message': 'Test db creation'},
 {'_id': ObjectId('5a6a0d61cd7a89005382c2d7'), 'message': 'Test db creation'},
 {'_id': ObjectId('5a6b6b7ecd7a8902168ae1d7'), 'message': 'Test db creation'}]

## Check content of collections

In [26]:
cursor=test_cllr.find().count()
cursor

10

In [27]:
loads_cllr=myWiki_dbr.loads_collection
cursor=loads_cllr.find()
cdocs=pd.DataFrame(list(cursor))
try:
    cdocs=cdocs.drop('_id',1)
except:
    cdocs
display(cdocs)
    

,loaded,master_cat,sub_cats_added
0,Thu Jan 25 22:20:35 2018,Category:Machine learning,0
1,Thu Jan 25 22:22:10 2018,Category:Machine learning,0
2,Thu Jan 25 22:31:22 2018,Category:Machine learning,50
3,Thu Jan 25 23:27:09 2018,Category:Business software,207


In [28]:
category_cllr=myWiki_dbr.category_collection
cursor=category_cllr.find().count()
display(cursor)

257

In [29]:
pagetest_cllr=myWiki_dbr.pagetest_collection
cursor=pagetest_cllr.find().count()
display(cursor)

255

In [31]:
page_cllr=myWiki_dbr.page_collection
cursor=page_cllr.find().count()
display(cursor)

5481

In [32]:
pagetestload_cllr=myWiki_dbr.pagetestload_collection
cursor=pagetestload_cllr.find()
doc=list(cursor)
pd.DataFrame(doc)

,_id,loaded,master_cat,pages_added
0,5a6a16d7cd7a8900a1f8496a,Thu Jan 25 17:41:43 2018,Category:Classification algorithms,255


In [33]:
pageload_cllr=myWiki_dbr.pageload_collection
cursor=pageload_cllr.find()
pdocs=pd.DataFrame(list(cursor))
try: 
    pdocs=pdocs.drop('_id',1)
except:
    pdocs
display(pdocs)

,loaded,master_cat,pages_added
0,Thu Jan 25 22:49:09 2018,Category:Machine learning,1130
1,Fri Jan 26 03:42:44 2018,Category:Business software,4351
